1. QFTの上記の実装は、$QFT^{\dag} \ket{\tilde{5}} = \ket{101}$であるフーリエ状態$\ket{\tilde{5}}$を準備することによってテストしました。 $QFT^{\dag} \ket{a} = \ket{100}$となるような状態$\ket{a}$を見つけてみてください。

A. 量子フーリエ変換の式は以下の通り。
$$QFT_N \ket{x} = \frac{1}{\sqrt{N}}(\ket{0} + \exp(\frac{2 \pi i}{2}x) \ket{1}) \otimes (\ket{0} + \exp(\frac{2 \pi i}{2^2}x) \ket{1})  \otimes ...  \otimes (\ket{0} + \exp(\frac{2 \pi i}{2^{n-1}}x) \ket{1}) \otimes (\ket{0} + \exp(\frac{2 \pi i}{2^n}x) \ket{1})$$

つまり、逆フーリエ変換をした際に左辺が$\ket{100}$となるような、右辺の状態を見つければよい。\
100を10進数表記に変換すると $1 \times 2^2 + 0 \times 2 ^ 1 + 0 \times 2 ^ 0 = 4$ なので $x = 4$ 、3量子ビットなので $n = 3、N = 2^3 = 8$ これらを上式の右辺に代入すると、
$$\frac{1}{\sqrt{8}}(\ket{0} + \exp(4\pi i) \ket{1}) \otimes (\ket{0} + \exp(\frac{4\pi i}{2}) \ket{1})  \otimes (\ket{0} + \exp(\frac{ 4\pi i}{4}) \ket{1})$$
となる。よって上記状態を量子回路にて作成していくと以下の通り。

In [ ]:
import numpy as np
from numpy import pi
# Qiskitをインポートする
from qiskit import QuantumCircuit, Aer, assemble
from qiskit.visualization import plot_bloch_multivector, plot_histogram

# 量子ビット数
nqubits = 3
# 逆フーリエ変換で導出したい状態x = 4
number = 4

qc = QuantumCircuit(nqubits)
# 各量子ビットにアダマールゲートを付与
for qubit in range(nqubits):
    qc.h(qubit)

# 各量子ビットにpゲートを付与
qc.p(number*pi/4,0)
qc.p(number*pi/2,1)
qc.p(number*pi,2)

qc.draw()

ブロッホ球で量子ビットの状態をチェックすると以下の通り。

In [ ]:
sim = Aer.get_backend("aer_simulator")
qc_init = qc.copy()
qc_init.save_statevector()
statevector = sim.run(qc_init).result().get_statevector()
plot_bloch_multivector(statevector)

上記で作成した状態に逆フーリエ変換すると以下の通り。

In [ ]:
def qft_rotations(circuit, n):
    # 回路の最初のn量子ビットでqftを実行する
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(pi/2**(n-qubit), qubit, n)

    # この関数の最後で、次の量子ビットで同じ関数を再度呼び出す（関数の前半でnを1つ減らしている）
    qft_rotations(circuit, n)

def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qft(circuit, n):
    # 回路の最初のn量子ビットでQFTを実行する
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

def inverse_qft(circuit, n):
    # 回路の最初のn量子ビットでInverse QFTを実行する

    # まず、n量子ビットのQFT回路を作成する
    qft_circ = qft(QuantumCircuit(n), n)
    # 次に、この回路の逆をとる
    invqft_circ = qft_circ.inverse()
    # 上で作成した逆QFT回路を既存の回路の最初のn量子ビットに追加する
    circuit.append(invqft_circ, circuit.qubits[:n])
    return circuit.decompose() # .decompose() で個々のゲートを確認できる

qc = inverse_qft(qc, nqubits)
qc.measure_all()

aer_sim = Aer.get_backend('aer_simulator')
qobj = assemble(qc)
result = aer_sim.run(qobj).result()
counts = result.get_counts()
plot_histogram(counts)

2. $QFT^{\dag} \ket{b} = \ket{011}$となるような状態$\ket{b}$を見つけてみてください。

A. 量子フーリエ変換の式は以下の通り。
$$QFT_N \ket{x} = \frac{1}{\sqrt{N}}(\ket{0} + \exp(\frac{2 \pi i}{2}x) \ket{1}) \otimes (\ket{0} + \exp(\frac{2 \pi i}{2^2}x) \ket{1})  \otimes ...  \otimes (\ket{0} + \exp(\frac{2 \pi i}{2^{n-1}}x) \ket{1}) \otimes (\ket{0} + \exp(\frac{2 \pi i}{2^n}x) \ket{1})$$

つまり、逆フーリエ変換をした際に左辺が$\ket{011}$となるような、右辺の状態を見つければよい。\
011を10進数表記に変換すると $0 \times 2^2  + 1 \times 2 ^ 1 + 1 \times 2 ^ 0 = 3$ なので $x = 3$ 、3量子ビットなので $n = 3、N = 2^3 = 8$ これらを上式の右辺に代入すると、
$$\frac{1}{\sqrt{8}}(\ket{0} + \exp(3\pi i) \ket{1}) \otimes (\ket{0} + \exp(\frac{3\pi i}{2}) \ket{1})  \otimes (\ket{0} + \exp(\frac{ 3\pi i}{4}) \ket{1})$$
となる。よって上記状態を量子回路にて作成していくと以下の通り。

In [ ]:
import numpy as np
from numpy import pi
# Qiskitをインポートする
from qiskit import QuantumCircuit, Aer, assemble
from qiskit.visualization import plot_bloch_multivector, plot_histogram

# 量子ビット数
nqubits = 3
# 逆フーリエ変換で導出したい状態x = 3
number = 3

qc = QuantumCircuit(nqubits)
# 各量子ビットにアダマールゲートを付与
for qubit in range(nqubits):
    qc.h(qubit)

# 各量子ビットにpゲートを付与
qc.p(number*pi/4,0)
qc.p(number*pi/2,1)
qc.p(number*pi,2)

qc.draw()

ブロッホ球で量子ビットの状態をチェックすると以下の通り。

In [ ]:
sim = Aer.get_backend("aer_simulator")
qc_init = qc.copy()
qc_init.save_statevector()
statevector = sim.run(qc_init).result().get_statevector()
plot_bloch_multivector(statevector)

上記で作成した状態に逆フーリエ変換すると以下の通り。

In [ ]:
def qft_rotations(circuit, n):
    # 回路の最初のn量子ビットでqftを実行する
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(pi/2**(n-qubit), qubit, n)

    # この関数の最後で、次の量子ビットで同じ関数を再度呼び出す（関数の前半でnを1つ減らしている）
    qft_rotations(circuit, n)

def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qft(circuit, n):
    # 回路の最初のn量子ビットでQFTを実行する
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

def inverse_qft(circuit, n):
    # 回路の最初のn量子ビットでInverse QFTを実行する

    # まず、n量子ビットのQFT回路を作成する
    qft_circ = qft(QuantumCircuit(n), n)
    # 次に、この回路の逆をとる
    invqft_circ = qft_circ.inverse()
    # 上で作成した逆QFT回路を既存の回路の最初のn量子ビットに追加する
    circuit.append(invqft_circ, circuit.qubits[:n])
    return circuit.decompose() # .decompose() で個々のゲートを確認できる

qc = inverse_qft(qc, nqubits)
qc.measure_all()

aer_sim = Aer.get_backend('aer_simulator')
qobj = assemble(qc)
result = aer_sim.run(qobj).result()
counts = result.get_counts()
plot_histogram(counts)

3. 再帰なしでQFT関数を記述してください。 Qiskitのユニタリーシミュレーターを使用して、結果を確認してください。

A.再起関数の代わりにforループを用いてQFT関数を実装する。コードは以下の通り。

In [ ]:
import numpy as np
from numpy import pi
from qiskit_textbook.widgets import scalable_circuit
# Qiskitをインポートする
from qiskit import QuantumCircuit, Aer, assemble

def qft_rotations(circuit, n):

    # qubit1のインデックスをn-1から0までループさせる
    for qubit1 in range(n-1, -1, -1):

        # qubit1にアダマールゲートを付与
        circuit.h(qubit1)

        # qubit1のインデックスが0になったらループを抜けてcircuitを返す
        if qubit1 == 0:
            return circuit

        # qubit2のインデックスを0からqubit1のインデックス-1までループさせ、それぞれCROTゲートを付与する
        for qubit2 in range(qubit1):
            circuit.cp(pi/2**(qubit1-qubit2), qubit2, qubit1)
    
def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qft(circuit, n):
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

scalable_circuit(qft)